https://huggingface.co/learn/audio-course/en/chapter4/fine-tuning (ref) <br/> 
https://huggingface.co/carlfeynman/carl-distilhubert-finetuned-gtzan (finetuned model)

In [ ]:
# !pip install datasets
# !pip install gradio
# !pip install accelerate -U

# !pip install soundfile
# !pip install librosa
# !pip install evaluate

# !pip install transformers -U

# !pip install --upgrade huggingface_hub
# !pip install -U gradio


In [ ]:
from datasets import load_dataset

In [ ]:
gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

Using custom data configuration all
Reusing dataset gtzan (/root/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/701395d625fb4762c27eef42c9488d843cc15869b399f7d207b07306a8b09762)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

In [ ]:
gtzan  = gtzan['train'].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/701395d625fb4762c27eef42c9488d843cc15869b399f7d207b07306a8b09762/cache-7848c6527387a015.arrow and /root/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/701395d625fb4762c27eef42c9488d843cc15869b399f7d207b07306a8b09762/cache-9b9f134c24d517c8.arrow


DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [ ]:
example = gtzan['train'][0]
example

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
  'array': array([ 0.10720825,  0.16122437,  0.28585815, ..., -0.22924805,
         -0.20629883, -0.11334229], dtype=float32),
  'sampling_rate': 22050},
 'genre': 7}

In [ ]:
id2label_fn = gtzan['train'].features['genre'].int2str

In [ ]:
id2label_fn(example['genre'])

'pop'

In [ ]:
import gradio as gr

In [ ]:
def sample_audio():
  example = gtzan['train'].shuffle()[0]
  audio = example['audio']
  return  (
      audio['sampling_rate'],
      audio['array']
  ), id2label_fn(example['genre'])

sample_audio()

((22050,
  array([ 0.00521851,  0.00061035,  0.0043335 , ..., -0.0149231 ,
         -0.01800537, -0.02783203], dtype=float32)),
 'country')

In [ ]:
# with gr.Blocks() as demo:
#   with gr.Column():
#     for _ in range(4):
#       audio, label = sample_audio()
#       output = gr.Audio(audio, label=label)
# demo.launch(debug=True)

In [ ]:
from transformers import AutoFeatureExtractor
from datasets import Audio

In [ ]:
model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id,
    do_normalize=True,
    return_attention_mask=True
)

In [ ]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [ ]:
gtzan = gtzan.cast_column('audio', Audio(sampling_rate=sampling_rate))

In [ ]:
gtzan['train'][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
  'array': array([ 0.0873509 ,  0.20183384,  0.4790867 , ..., -0.18743178,
         -0.23294401, -0.13517427], dtype=float32),
  'sampling_rate': 16000},
 'genre': 7}

In [ ]:
import numpy as np

In [ ]:
audio = gtzan['train'][0]['audio']
audio

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
 'array': array([ 0.0873509 ,  0.20183384,  0.4790867 , ..., -0.18743178,
        -0.23294401, -0.13517427], dtype=float32),
 'sampling_rate': 16000}

In [ ]:
print(f"mean: {np.mean(audio['array']):.3f} var: {np.var(audio['array']):.3f}")

mean: 0.000 var: 0.049


In [ ]:
inputs = feature_extractor(audio['array'], sampling_rate=audio['sampling_rate'])
print(f"mean: {np.mean(inputs['input_values']):.3f} var: {np.var(inputs['input_values']):.3f}")


mean: -0.000 var: 1.000


In [ ]:
max_duration = 30.0

def preprocess_audio(examples):
  audio_arr = [ele['array'] for ele in examples['audio']]
  inputs = feature_extractor(
      audio_arr,
      sampling_rate = feature_extractor.sampling_rate,
      max_length = int(feature_extractor.sampling_rate*max_duration),
      turncation = True,
      return_attention_mask = True
  )
  return inputs


In [ ]:
gtzan_encoded = gtzan.map(preprocess_audio, remove_columns=["audio", "file"],
          batched=True, batch_size=100, num_proc=1)
gtzan_encoded

Loading cached processed dataset at /root/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/701395d625fb4762c27eef42c9488d843cc15869b399f7d207b07306a8b09762/cache-317fb56431c4abe1.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/701395d625fb4762c27eef42c9488d843cc15869b399f7d207b07306a8b09762/cache-142a76ce28440c40.arrow


DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

In [ ]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")
gtzan_encoded

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

In [ ]:
gtzan_encoded['train'].features['label'].names

['blues',
 'classical',
 'country',
 'disco',
 'hiphop',
 'jazz',
 'metal',
 'pop',
 'reggae',
 'rock']

In [ ]:
id2label = {
    str(i): id2label_fn(i) for i in range(len(gtzan_encoded['train'].features['label'].names))
}
label2id = {l:i for i,l in  id2label.items()}
id2label['7']

'pop'

In [ ]:
from transformers import AutoModelForAudioClassification

In [ ]:
num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TrainingArguments

In [ ]:
model_name = model_id.split('/')[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 5

In [ ]:
training_args = TrainingArguments(
    f"carl-{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
    dataloader_num_workers=5,
    dataloader_prefetch_factor=5,
    report_to=None
)

In [ ]:
import evaluate

In [ ]:
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
  preds = np.argmax(eval_pred.predictions, axis=1)
  return metric.compute(predictions=preds, references=eval_pred.label_ids)

In [ ]:
from transformers import Trainer

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.794400,1.720942,0.520000
2,1.224700,1.185684,0.750000
3,1.047700,0.969656,0.740000
4,0.741800,0.860538,0.780000
5,0.659500,0.769503,0.810000


Checkpoint destination directory carl-distilhubert-finetuned-gtzan/checkpoint-113 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=565, training_loss=1.2444716554827395, metrics={'train_runtime': 424.6266, 'train_samples_per_second': 10.586, 'train_steps_per_second': 1.331, 'total_flos': 3.0785066831834835e+17, 'train_loss': 1.2444716554827395, 'epoch': 5.0})

In [ ]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"carl-{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

In [ ]:
trainer.push_to_hub(**kwargs)

CommitInfo(commit_url='https://huggingface.co/carlfeynman/carl-distilhubert-finetuned-gtzan/commit/b80472f6c6b9299202836814223b3f78b1a64363', commit_message='End of training', commit_description='', oid='b80472f6c6b9299202836814223b3f78b1a64363', pr_url=None, pr_revision=None, pr_num=None)